In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('processed_data_5000.csv')
data.head(10)

,content
0,Kumaha barudak? Well!Yuk ramaikan Pesta Anak N...
1,Kustini Sri Purnomo memiliki strategi jitu unt...
2,"Peraih Emas Olimpiade Tokyo, Greysia Polii bic..."
3,Kapolri Jenderal Lisyto Sigit Prabowo bicara a...
4,"Menteri BUMN, Erick Thohir bicara bangsa Indon..."
5,Iring-iringan mobil SUV pasukan pengawal Presi...
6,Seorang remaja Palestina umur 14 tahun ditemba...
7,"Ketua Umum PDIP, Megawati Soekarnoputri mengin..."
8,Sekjen PDIP Hasto Kristiyanto berbicara terkai...
9,Presiden Joko Widodo memimpin upacara peringat...


In [3]:
from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline

# Load model analisis sentimen bahasa Indonesia dengan tiga kelas sentimen
sentiment_analysis = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")


2024-04-17 11:34:06.830749: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 11:34:08.282904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Fungsi untuk melabeli sentimen pada setiap baris data
def label_sentiment(text):
    # Analisis sentimen menggunakan model Transformer
    result = sentiment_analysis(text)
    # Ambil label sentimen teratas
    top_sentiment = result[0]
    label = top_sentiment['label']
    score = top_sentiment['score']
    print(text,label)
    return label, score

# Fungsi untuk melabeli sentimen menggunakan multi-threading
def label_sentiment_multithread(text_list):
    with ThreadPoolExecutor() as executor:
        results = executor.map(label_sentiment, text_list)
    return list(results)

# Bagi data menjadi batch agar dapat diproses menggunakan multi-threading
batch_size = 1000
num_batches = len(data) // batch_size + 1
batches = [data[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]

# Melabeli sentimen untuk setiap batch menggunakan multi-threading
labeled_data = []
for batch in batches:
    texts = batch['content'].tolist()
    labels = label_sentiment_multithread(texts)
    labeled_data.extend(labels)

# Memasukkan hasil label sentimen ke dalam DataFrame
data['sentiment_label'], data['sentiment_score'] = zip(*labeled_data)

Presiden Joko Widodo memimpin upacara peringatan Hari Kesaktian Pancasila di Lubang Buaya, Pondok Gede, Jakarta Timur, Sabtu (1/10). Upacara berlangsung dengan khidmat. Neutral
Ketua Umum PDIP, Megawati Soekarnoputri mengingatkan seluruh kader untuk menyatu dengan rakyat. Hal itu dilakukan untuk memenangkan Ganjar Pranowo di pilpres 2024. Neutral
Menteri BUMN, Erick Thohir bicara bangsa Indonesia membutuhkan pribadi-pribadi yang siap melangkah lebih besar dan konsepsi yang lebih matang untuk mewujudkan visi Indonesia emas. Ia pun mengajak anak muda Indonesia untuk terus maju dan bergerak demi Indonesia yang lebih baik. Neutral
Iring-iringan mobil SUV pasukan pengawal Presiden AS Joe Biden ditabrak sebuah mobil. Presiden Biden dan Ibu Negara Jill Biden dipastikan selamat. Neutral
Seorang remaja Palestina umur 14 tahun ditembak mati polisi Israel karena melakukan penikaman di Yerusalem, Rabu (30/8) malam. Korban warga Israel berusia 25 tahun mengalami luka tusuk di punggung. Neutral
Kapo

In [ ]:
# Simpan data yang telah dilabeli ke dalam file CSV
data.to_csv('labeled_3.csv', index=False)